# 第一部分 抓取住建局数据并保存
## 1.保证Chrome和ChromeDriver版本匹配
## 2.设置ChromeDriver的路径
## 3.设置楼盘的关键词列表
## 4.设置csv的保存路径

In [ ]:
import time
import csv
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from collections import defaultdict

# 记录开始时间
start_time = time.time()

# 设置Chrome选项，注意保证Chrome和ChromeDriver版本匹配
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 指定ChromeDriver路径
service = Service('/usr/local/bin/chromedriver')  # ChromeDriver的路径，注意替换
driver = webdriver.Chrome(service=service, options=chrome_options)

# 访问目标网页
driver.get('https://zw.cdzjryb.com/zwdt/SCXX/Default.aspx?action=ucSCXXShowNew')

# 等待页面加载
wait = WebDriverWait(driver, 10)

# 关键词列表，替换成想关注的楼盘，注意名字是住建局的备案名，可以在上面的网址进行确认
keywords = [
    "新川锦萃名邸", "蜀道宽境小区"
]

# 用来保存所有链接的列表
urls = []

# 存储数据的列表
data_list = []

# 遍历关键词列表
for keyword in keywords:
    input_field = wait.until(EC.presence_of_element_located((By.ID, 'ID_ucSCXXShowNew_txtpName')))
    input_field.clear()
    input_field.send_keys(keyword)

    search_button = driver.find_element(By.ID, 'ID_ucSCXXShowNew_btnSccx')
    search_button.click()

    time.sleep(3)
    
    elements = driver.find_elements(By.XPATH, "//a[starts-with(@href, 'https://zw.cdzjryb.com/roompricezjw/index.html?param=')]")
    
    for element in elements:
        urls.append({
            "url": element.get_attribute('href'),
            "keyword": keyword
        })

print(f"提取的链接总数: {len(urls)}")

for item in urls:
    url = item["url"]
    keyword = item["keyword"]

    print(f"正在爬取: {url}")
    driver.get(url)

    retries = 3
    for attempt in range(retries):
        try:
            building_elements = WebDriverWait(driver, 7).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.rp-nav-item'))
            )

            for building in building_elements:
                building_num = building.get_attribute('data-val')
                print(f"正在抓取楼栋: {building_num}")

                driver.execute_script("arguments[0].click();", building)

                unit_elements = WebDriverWait(driver, 3).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, f'.rp-subnav-item[data-parentval="{building_num}"]'))
                )

                for unit in unit_elements:
                    unit_num = unit.get_attribute('data-val')
                    print(f"抓取单元: {unit_num}")

                    driver.execute_script("arguments[0].click();", unit)

                    WebDriverWait(driver, 5).until(
                        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.tbl-room tbody tr'))
                    )

                    rows = driver.find_elements(By.CSS_SELECTOR, '.tbl-room tbody tr')
                    for row in rows:
                        columns = row.find_elements(By.TAG_NAME, 'td')
                        if columns:
                            area_raw = columns[3].text
                            area_approx = round(float(area_raw)) if area_raw else None

                            sale_status = columns[8].text
                            if sale_status == "可售":
                                sale_situation = "待售"
                            elif sale_status in ["拟定合同", "已售"]:
                                sale_situation = "售出"
                            else:
                                sale_situation = "检查数据"

                            data = {
                                "小区": keyword,
                                "楼栋": building_num,
                                "单元": unit_num,
                                "楼栋号": columns[0].text,
                                "楼层": columns[1].text,
                                "房号": columns[2].text,
                                "建筑面积（㎡）": area_raw,
                                "建筑面积（近似值）": area_approx,
                                "户型": columns[4].text,
                                "挂牌清水价（元）": columns[5].text,
                                "装修价格（元）": columns[6].text,
                                "房屋用途": columns[7].text,
                                "销售状态": sale_status,
                                "销售情况": sale_situation,
                                "抵押": columns[9].text,
                                "查封": columns[10].text,
                                "数据来源": url,
                                "爬取日期": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                            }
                            if data["房屋用途"] not in ["车位", "商业"]:
                                data_list.append(data)

            break

        except (StaleElementReferenceException, TimeoutException) as e:
            print(f"遇到错误: {e}. 尝试重新加载页面, 当前尝试: {attempt + 1}/{retries}")
            driver.refresh()
            time.sleep(3)

            if attempt == retries - 1:
                print(f"无法爬取 {url}，程序终止。")
                driver.quit()
                raise Exception(f"爬取失败：{url} 超过最大重试次数。")

driver.quit()

# 去重处理
seen = set()
unique_data_list = []
for item in data_list:
    identifier = (item["小区"], item["楼栋"], item["单元"], item["楼栋号"], item["楼层"], item["房号"], item["建筑面积（㎡）"], item["户型"], item["挂牌清水价（元）"], item["装修价格（元）"], item["房屋用途"], item["销售状态"], item["销售情况"], item["抵押"], item["查封"], item["数据来源"])
    if identifier not in seen:
        seen.add(identifier)
        unique_data_list.append(item)

# 保存去重后的数据到CSV文件，注意替换保存路径
output_file = "文件路径/成都楼盘销售情况爬取/csv存档/成都楼盘销售情况爬取_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".csv"
with open(output_file, mode='w', newline='', encoding='utf-8-sig') as file:
    fieldnames = ["小区", "楼栋", "单元", "楼栋号", "楼层", "房号", "建筑面积（㎡）", "建筑面积（近似值）", "户型", "挂牌清水价（元）", "装修价格（元）", "房屋用途", "销售状态", "销售情况", "抵押", "查封", "数据来源", "爬取日期"]
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(unique_data_list)

print(f"数据已成功保存到 {output_file}")

# 初始化汇总数据的结构
summary = defaultdict(lambda: {
    "总房源": 0,
    "售出": 0,
    "待售": 0,
    "面积汇总": defaultdict(lambda: {"售出": 0, "待售": 0})
})

for row in unique_data_list:
    community = row["小区"]
    area = int(float(row["建筑面积（近似值）"]))
    sale_situation = row["销售情况"]

    summary[community]["总房源"] += 1
    summary[community][sale_situation] += 1
    summary[community]["面积汇总"][area][sale_situation] += 1

print("\n########################################\n")
print(f"截至{datetime.now().strftime('%Y年%m月%d日%H点%M分')},")
for community, stats in summary.items():
    print(f"{community}共挂出{stats['总房源']}套房源，售出{stats['售出']}套、待售{stats['待售']}套。")
    
    for area in sorted(stats["面积汇总"].keys()):
        sale_stats = stats["面积汇总"][area]
        print(f"{area}面积：售出{sale_stats['售出']}套、待售{sale_stats['待售']}套")
    
    print("*" * 40)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"程序运行时间: {elapsed_time:.2f} 秒")


# 第二部分 生成当前楼盘销售情况表格
## 1.设置表格字体防止乱码

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import os
import random
from datetime import datetime

# 全局设置字体为PingFang字体，Windows下注意替换字体
plt.rcParams['font.family'] = 'PingFang HK'  # macOS下中文字体
plt.rcParams['axes.unicode_minus'] = False  # 避免显示为方块

# 随机生成低饱和度的颜色
def random_light_color():
    r = random.uniform(0.7, 0.9)
    g = random.uniform(0.7, 0.9)
    b = random.uniform(0.7, 0.9)
    return (r, g, b)

# 初始化图表
fig, ax = plt.subplots(figsize=(10, 10))  # 调整图表大小
ax.set_axis_off()  # 隐藏轴

# 初始化总文本数据
y_position = 1.0  # 控制每个小区的位置，从顶部开始
spacing = 0.05  # 小区之间的间距


# 汇总的数据
for community, stats in summary.items():
    # 添加总房源、售出、待售的文字描述
    total_text = f"{community}共挂出房源{stats['总房源']}套，售出{stats['售出']}套，待售{stats['待售']}套。"
    
    # 在图中插入小区描述文字，y_position 控制纵向位置
    ax.text(0, y_position, total_text, fontsize=12, va='top')
    y_position -= spacing  # 在插入文字后，调整下一个表格的起始位置

    # 准备小区的面积、售出、待售数据
    data = []
    for area, sale_stats in sorted(stats["面积汇总"].items()):
        data.append({
            "面积": f"{area} ㎡",
            "售出数量": sale_stats["售出"],
            "待售数量": sale_stats["待售"]
        })

    # 创建小区的数据框
    df = pd.DataFrame(data)

    # 随机选择一个低饱和度的颜色用于表头
    header_color = random_light_color()

    # 计算表格高度，根据行数动态调整
    num_rows = len(df)
    table_height = 0.05 * num_rows  # 每行的高度根据行数动态调整

    # 添加表格
    table = ax.table(cellText=df.values, colLabels=df.columns, loc='center', cellLoc='center',
                     colColours=[header_color] * len(df.columns), bbox=[0, y_position-table_height, 1, table_height])

    # 设置表格字体
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)

    # 更新 y_position 位置，确保下一个小区有足够的空间
    y_position -= table_height + spacing

# 添加生成时间作为标题
current_time = datetime.now().strftime("%Y年%m月%d日%H点%M分")
fig.suptitle(f"楼盘销售情况（截至{current_time}）", fontsize=14)

# 保存图片的目录，替换成文件路径
output_dir = "文件路径/成都楼盘销售情况爬取/"
os.makedirs(output_dir, exist_ok=True)  # 确保目录存在，必要时创建

# 保存图片
output_file = os.path.join(output_dir, f"楼盘销售情况_{datetime.now().strftime('%Y%m%d_%H%M%S')}.png")
plt.savefig(output_file, bbox_inches='tight', dpi=300)

print(f"表格图片已保存到 {output_file}")


# 第三部分 生成销售情况对比表（汇总一段时间内，各楼盘、各户型的销售情况）
## 1.提供csv文件夹路径（在第一部分设置的那个）
## 2.设置图片存档路径

In [ ]:
import pandas as pd
import re
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os

# 设置文件夹路径，为第一部分存储csv文件的文件夹
folder_path = '文件路径/成都楼盘销售情况爬取/csv存档/'

# 获取最新的两个 CSV 文件
def get_latest_csv_files(folder_path):
    files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    files.sort(key=lambda x: re.search(r'_(\d{8})_', x).group(1), reverse=True)
    return [os.path.join(folder_path, files[i]) for i in range(2)]

# 获取最新的两个 CSV 文件路径
file_new, file_old = get_latest_csv_files(folder_path)

# 提取日期
def extract_date_from_filename(filename):
    match = re.search(r'_(\d{8})_', filename)
    if match:
        return datetime.strptime(match.group(1), '%Y%m%d').date()
    return None

date_new = extract_date_from_filename(file_new)
date_old = extract_date_from_filename(file_old)

# 读取 CSV 文件
df_new = pd.read_csv(file_new)
df_old = pd.read_csv(file_old)

# 数据清洗: 删除重复项和空值
df_new.drop_duplicates(inplace=True)
df_old.drop_duplicates(inplace=True)

# 汇总各个小区、各个面积的数量
new_summary = df_new.groupby(['小区', '建筑面积（近似值）']).size().reset_index(name='数量')
old_summary = df_old.groupby(['小区', '建筑面积（近似值）']).size().reset_index(name='数量')

# 合并汇总数据
summary = pd.merge(new_summary, old_summary, on=['小区', '建筑面积（近似值）'], how='outer', suffixes=('_新', '_旧')).fillna(0)

# 计算新增房源
summary['新增房源'] = summary['数量_新'] - summary['数量_旧']

# 获取新增房源的详细信息
new_houses = summary[summary['新增房源'] > 0]

# 汇总新售出房源情况，基于“销售情况”列
sold_summary_new = df_new[df_new['销售情况'] == '售出'].groupby(['小区', '建筑面积（近似值）']).size().reset_index(name='新售出数量')
sold_summary_old = df_old[df_old['销售情况'] == '售出'].groupby(['小区', '建筑面积（近似值）']).size().reset_index(name='售出数量')

# 合并售出汇总数据
sold_summary = pd.merge(sold_summary_new, sold_summary_old, on=['小区', '建筑面积（近似值）'], how='outer', suffixes=('_新', '_旧')).fillna(0)

# 计算新售出房源
sold_summary['新售出数量'] = sold_summary['新售出数量'].fillna(0)
sold_summary['售出数量'] = sold_summary['售出数量'].fillna(0)
sold_summary['新增售出'] = sold_summary['新售出数量'] - sold_summary['售出数量']

# 获取新增售出房源的详细信息
sold_houses = sold_summary[sold_summary['新增售出'] > 0]

# 汇总每个小区的新增售出总数
total_sold = sold_houses.groupby('小区')['新增售出'].sum().reset_index()

# 添加汇总行到 sold_houses
summary_with_totals = []
color_map = ['#D8BFD8', '#E6E6FA', '#FFE4E1', '#FFB6C1', '#D3D3D3']  # 莫兰迪色系
current_color_index = 0

for name, group in sold_houses.groupby('小区'):
    color = color_map[current_color_index % len(color_map)]
    summary_with_totals.append(group[['小区', '建筑面积（近似值）', '新增售出']].rename(columns={'建筑面积（近似值）': '面积'}))

    # 添加汇总行
    total_row = total_sold[total_sold['小区'] == name]
    if not total_row.empty:
        summary_with_totals.append(pd.DataFrame({
            '小区': [name],
            '面积': ['总计'],
            '新增售出': [total_row['新增售出'].values[0]]
        }))
        
    current_color_index += 1

final_summary = pd.concat(summary_with_totals, ignore_index=True)

# 生成PNG表格的函数
def save_table_as_png(df, filename, title):
    fig, ax = plt.subplots(figsize=(10, 2 + len(df) * 0.3))  # 设置表格大小
    ax.axis('tight')
    ax.axis('off')

    # 创建表格
    table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

    # 设置行和列的颜色
    current_color_index = 0
    for i in range(len(df)):
        color = color_map[current_color_index % len(color_map)]
        for j in range(len(df.columns)):
            table[i + 1, j].set_facecolor(color)
        if i < len(df) - 1 and df.loc[i + 1, '小区'] != df.loc[i, '小区']:
            current_color_index += 1

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.2)
    plt.title(title)
    plt.savefig(filename, bbox_inches='tight', dpi=300)
    plt.close()

# 文件保存目录和命名
save_path = output_dir
file_name = f'新增出售情况_{date_old}到{date_new}.png'

# 保存表格
if not final_summary.empty:
    save_table_as_png(final_summary, f'{save_path}/{file_name}', f'新售出房源 ({date_old} 到 {date_new})')

# 在Jupyter Notebook中显示PNG图像
from IPython.display import Image, display
display(Image(filename=f'{save_path}/{file_name}'))

print("表格已保存为PNG格式，并在Notebook中显示。")
